
## Simd examples


First of all, to ensure that you can use simd instructions, run 
```
include(joinpath(dirname(JULIA_HOME),"share","julia","build_sysimg.jl")); build_sysimg(force=true)
```
if julia is not build in source.

- https://discourse.julialang.org/t/does-julia-use-simd-instructions-for-broadcast-operations/2492/18


### Making operations in a big array

In [1]:
using BenchmarkTools

┌ Info: Recompiling stale cache file /Users/davidbuchacaprats/.julia/compiled/v1.1/BenchmarkTools/ZXPQo.ji for BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1184


In [2]:
function mysum(a::Vector)
    total = zero(eltype(a))
    for x in a
        total += x
    end
    return total
end

function mysum_simd(a::Vector)
    total = zero(eltype(a))
    @simd for x in a
        total += x
    end
    return total
end

mysum_simd (generic function with 1 method)

#### @simd macro

When we use the `@simd` macro the for loop gets changed by a set a couple of loops. The main idea is that operations that are done "sequentially" can be done at the sime time for a given simd width.


Instead of doing: `total = 0; total += x[1]; ... ;total += x[n] `. We can do the same operation (in this case `+`) on `n_simd` elements at the same time (for the same cost we can do a single `+` operation). If we had 10 elements and  `n_simd=4` we could make 3 sums (instead of 10) followed by a sum of the 3 partial results. The partial results would contain the partial sums, one from `x[1:4]`, another from `x[4:8]` and the last one from `x[8:10]`.

In this case it is likely that the overhead produced by the `@simd` is not worth the effort. Nevertheless, these instructions become paramount when operations on arrays of thousands of elements.

We can see the internals of `@simd` macro in the next cell:

In [3]:
@macroexpand @simd for x in a
        total += x
    end

quote
    #= simdloop.jl:65 =#
    let ##r#367 = a
        #= simdloop.jl:66 =#
        for ##i#368 = Base.simd_outer_range(##r#367)
            #= simdloop.jl:67 =#
            let ##n#369 = Base.simd_inner_length(##r#367, ##i#368)
                #= simdloop.jl:68 =#
                if zero(##n#369) < ##n#369
                    #= simdloop.jl:70 =#
                    let ##i#370 = zero(##n#369)
                        #= simdloop.jl:71 =#
                        while ##i#370 < ##n#369
                            #= simdloop.jl:72 =#
                            local x = Base.simd_index(##r#367, ##i#368, ##i#370)
                            #= simdloop.jl:73 =#
                            begin
                                #= In[3]:2 =#
                                total += x
                            end
                            #= simdloop.jl:74 =#
                            ##i#370 += 1
                            #= simdloop.jl:75 =#
                            $(Ex

#### Float type impact

The following benchmarks show that

- `mysum_simd` is is twice faster with `Float32` than with `Float64`.
- `mysum_simd` and `mysum` are much slower with `Float16`than with `Float32` and `Float64` (probably because there are not `Float16` instructions called in this processor).


The test found in the next cell can give  you different results if you installed julia from source or not. You can use the provided script with the command `julia execute_this.jl` to build it from source.

```julia

# with execute_this.jl executed
mysum_simd      float64:    Trial(10.411 μs)
mysum           float64:    Trial(89.152 μs)

mysum_simd      float32:    Trial(5.035 μs)
mysum           float32:    Trial(89.168 μs)

mysum_simd      float16:    Trial(846.852 μs)
mysum           float16:    Trial(820.405 μs)


# with execute_this.jl executed
mysum_simd      float64:    Trial(22.340 μs)
mysum           float64:    Trial(89.161 μs)

mysum_simd      float32:    Trial(11.210 μs)
mysum           float32:    Trial(89.170 μs)

mysum_simd      float16:    Trial(817.455 μs)
mysum           float16:    Trial(817.467 μs)
```


In [4]:
# with execute_this.jl executed
x = rand(Float64 , 100000);
println("mysum_simd      float64:    ", @benchmark mysum_simd(x))
println("mysum           float64:    ", @benchmark mysum(x))

x = rand(Float32 , 100000);
println()
println("mysum_simd      float32:    ", @benchmark mysum_simd(x))
println("mysum           float32:    ", @benchmark mysum(x))

println()
x = rand(Float16 , 100000);
println("mysum_simd      float16:    ", @benchmark mysum_simd(x))
println("mysum           float16:    ", @benchmark mysum(x))

mysum_simd      float64:    Trial(23.326 μs)
mysum           float64:    Trial(83.620 μs)

mysum_simd      float32:    Trial(11.154 μs)
mysum           float32:    Trial(83.614 μs)

mysum_simd      float16:    Trial(811.922 μs)
mysum           float16:    Trial(845.870 μs)


In [4]:
#@code_native mysum_simd(x)

In [5]:
#@code_native mysum(x)

### Example cost function simd




In [6]:
function MSE_simd{T}(y::Vector{T},y_pred::Vector{T})
   cost = zero(eltype(y))
   @simd for i in 1:length(y)
       @inbounds cost += (y[i] - y_pred[i])^2
   end
   return sqrt(cost)
end

function MSE{T}(y::Vector{T},y_pred::Vector{T})
   cost = zero(eltype(y))
    
    @inbounds for i in 1:length(y)
        cost += (y[i] - y_pred[i])^2
    end
    return sqrt(cost)
end

MSE (generic function with 1 method)

In [7]:
y     =  rand(Float64 , 100000);
y_hat =  rand(Float64 , 100000);
println("MSE             float64:    ", @benchmark MSE(y,y_hat))
println("MSE_simd        float64:    ", @benchmark MSE_simd(y,y_hat))

y     =  rand(Float32 , 100000);
y_hat =  rand(Float32 , 100000);
println()
println("MSE             float32:    ", @benchmark MSE(y,y_hat))
println("MSE_simd        float32:    ", @benchmark MSE_simd(y,y_hat))

y     =  rand(Float16 , 100000);
y_hat =  rand(Float16 , 100000);
println()
println("MSE             float16:    ", @benchmark MSE(y,y_hat))
println("MSE_simd        float16:    ", @benchmark MSE_simd(y,y_hat))

MSE             float64:    Trial(89.164 μs)
MSE_simd        float64:    Trial(19.770 μs)

MSE             float32:    Trial(89.176 μs)
MSE_simd        float32:    Trial(9.935 μs)

MSE             float16:    Trial(2.720 ms)
MSE_simd        float16:    Trial(2.719 ms)


### cross entropy minibatch

In [15]:
T = Float32
p_y_given_x = rand(T, 10, 256);
onehot_y = zeros(T, 10, 256);

In [16]:
onehot_y

10×256 Array{Float32,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

#### Another exmaple

- https://discourse.julialang.org/t/speeding-up-fvm-code/8430/7

In [8]:
function rhs!(divergence,Nv,Nw,invDv,invDw,g,V,W)
    # this is type stable!!
    @inbounds for i=1:Nv
        for j= Nw:-1:1
            # Values
            gij  = g[j,i]
            gimj = i > 1         ? g[j,i-1]:zero(gij)
            gipj = (i+1) <= Nv   ? g[j,i+1]:zero(gij)

            gijp = j+1 < Nw      ? g[j+1,i]:zero(gij)
            gijm = j-1 >= 1      ? g[j-1,i]:zero(gij)
            # Upwind fluxes + null flux boundary conditions
            Fij  = i > 1         ? (V[j,i]>=0.  ? V[j,i]*gimj: V[j,i]  *gij): zero(gij)
            Fipj = i < Nv        ? (V[j,i+1]>=0.? V[j,i+1]*gij:V[j,i+1]*gipj):zero(gij)

            Gijp = j < Nw        ? (W[j+1,i]>0. ? W[j+1,i]*gij:W[j+1,i]*gijp):zero(gij)
            Gij  = j>1           ? (W[j,i]>0.   ? W[j,i]*gijm: W[j,i]  *gij): zero(gij)
            # Divergence
            divergence[j,i] = invDv*(Fipj - Fij) + invDw*(Gijp - Gij)
        end
    end
end

N = 2000
divergence = rand(N,N)
g = rand(N,N)
V = rand(N,N)
W = rand(N,N)
@benchmark rhs!(divergence,N,N,1.,1.,g,V,W)


BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     16.898 ms (0.00% GC)
  median time:      17.038 ms (0.00% GC)
  mean time:        17.124 ms (0.00% GC)
  maximum time:     19.012 ms (0.00% GC)
  --------------
  samples:          292
  evals/sample:     1

In [11]:

function inner(i, divergence,Nv,Nw,invDv,invDw,g,V,W)
    @inbounds for j= Nw:-1:1
        # Values
        gij  = g[j,i]
        gimj = i > 1         ? g[j,i-1]:zero(gij)
        gipj = (i+1) <= Nv   ? g[j,i+1]:zero(gij)

        gijp = j+1 < Nw      ? g[j+1,i]:zero(gij)
        gijm = j-1 >= 1      ? g[j-1,i]:zero(gij)
        # Upwind fluxes + null flux boundary conditions
        Fij  = i > 1         ? (V[j,i]>=0.  ? V[j,i]*gimj: V[j,i]  *gij): zero(gij)
        Fipj = i < Nv        ? (V[j,i+1]>=0.? V[j,i+1]*gij:V[j,i+1]*gipj):zero(gij)

        Gijp = j < Nw        ? (W[j+1,i]>0. ? W[j+1,i]*gij:W[j+1,i]*gijp):zero(gij)
        Gij  = j>1           ? (W[j,i]>0.   ? W[j,i]*gijm: W[j,i]  *gij): zero(gij)
        # Divergence
        divergence[j,i] = invDv*(Fipj - Fij) + invDw*(Gijp - Gij)
    end
end
function rhs!(divergence,Nv,Nw,invDv,invDw,g,V,W)
    # this is type stable!!
    Threads.@threads for i=1:Nv
        inner(i, divergence,Nv,Nw,invDv,invDw,g,V,W)
    end
end

using BenchmarkTools
function init()
    srand(42)
    N = 2000
    divergence = rand(N,N)
    g = rand(N,N)
    V = rand(N,N)
    W = rand(N,N)
    divergence, g, V, W
end
N = 2000
@benchmark rhs!(divergence,N,N,1.,1.,g,V,W) setup= ((divergence, g, V, W) = init())


BenchmarkTools.Trial: 
  memory estimate:  96 bytes
  allocs estimate:  1
  --------------
  minimum time:     19.779 ms (0.00% GC)
  median time:      20.044 ms (0.00% GC)
  mean time:        20.238 ms (0.00% GC)
  maximum time:     21.451 ms (0.00% GC)
  --------------
  samples:          78
  evals/sample:     1

In [10]:
function inner(i, divergence,Nv,Nw,invDv,invDw,g,V,W)
    @simd for j = Nw-1:-1:2
        @inbounds begin
            # Values
            gij  = g[j, i]
            gimj = g[j, i-1]
            gipj = g[j, i+1]

            gijp = g[j+1, i]
            gijm = g[j-1, i]

            # Upwind fluxes + null flux boundary conditions
            vji = V[j,i]
            mul1 = ifelse(vji >= 0.0, gimj, gij)
            Fij  = vji * mul1
            
            vji = V[j,i+1]
            mul1 = ifelse(vji >= 0.0, gij, gipj)
            Fipj  = vji * mul1
            
            wji = W[j+1,i]
            mul = ifelse(wji > 0.0, gij, gijp)
            Gijp  = wji * mul
            
            wji = W[j,i]
            mul = ifelse(wji > 0.0, gijm, gij)
            Gij  = wji * mul

            # Divergence
            divergence[j,i] = invDv*(Fipj - Fij) + invDw*(Gijp - Gij)
        end
    end
end
function rhs!(divergence,Nv,Nw,invDv,invDw,g,V,W)
    Threads.@threads for i=2:Nv-1
        inner(i, divergence,Nv,Nw,invDv,invDw,g,V,W)
    end
end
using BenchmarkTools
function init()
    srand(42)
    N = 2000
    divergence = rand(N,N)
    g = rand(N,N)
    V = rand(N,N)
    W = rand(N,N)
    divergence, g, V, W
end
N = 2000
#@btime rhs!(divergence,N,N,1.,1.,g,V,W) setup= ((divergence, g, V, W) = init())
@benchmark rhs!(divergence,N,N,1.,1.,g,V,W) setup= ((divergence, g, V, W) = init())



BenchmarkTools.Trial: 
  memory estimate:  96 bytes
  allocs estimate:  1
  --------------
  minimum time:     8.527 ms (0.00% GC)
  median time:      8.804 ms (0.00% GC)
  mean time:        8.953 ms (0.00% GC)
  maximum time:     10.175 ms (0.00% GC)
  --------------
  samples:          94
  evals/sample:     1

### median pooling

In [13]:
zero(Float32)

0.0f0

In [46]:
#https://discourse.julialang.org/t/make-this-code-fast-median-pooling/6405

In [8]:
@inline function median5_swap(a,b,c,d,e)
    # https://github.com/JeffreySarnoff/SortingNetworks.jl/blob/master/src/swapsort.jl
    a,b = minmax(a,b)
    c,d = minmax(c,d)
    a,c = minmax(a,c)
    b,d = minmax(b,d)
    c,e = minmax(e,c)
    max(c, min(e,b))
end

@inline median5(args...) = median5_swap(args...)

function medmedpool55!(out::AbstractMatrix, img::AbstractMatrix)
    @assert size(out, 1) >= size(img, 1) ÷ 5
    @assert size(out, 2) >= size(img, 2) ÷ 5
    @inbounds for j ∈ indices(out)[2]
        @simd for i ∈ indices(out)[1]
            x11 = img[5i-4, 5j-4]
            x21 = img[5i-3, 5j-4]
            x31 = img[5i-2, 5j-4]
            x41 = img[5i-1, 5j-4]
            x51 = img[5i-0, 5j-4]
            
            x12 = img[5i-4, 5j-3]
            x22 = img[5i-3, 5j-3]
            x32 = img[5i-2, 5j-3]
            x42 = img[5i-1, 5j-3]
            x52 = img[5i-0, 5j-3]
            
            x13 = img[5i-4, 5j-2]
            x23 = img[5i-3, 5j-2]
            x33 = img[5i-2, 5j-2]
            x43 = img[5i-1, 5j-2]
            x53 = img[5i-0, 5j-2]
            
            x14 = img[5i-4, 5j-1]
            x24 = img[5i-3, 5j-1]
            x34 = img[5i-2, 5j-1]
            x44 = img[5i-1, 5j-1]
            x54 = img[5i-0, 5j-1]
            
            x15 = img[5i-4, 5j-0]
            x25 = img[5i-3, 5j-0]
            x35 = img[5i-2, 5j-0]
            x45 = img[5i-1, 5j-0]
            x55 = img[5i-0, 5j-0]
            
            y1 = median5(x11,x12,x13,x14,x15)
            y2 = median5(x21,x22,x23,x24,x25)
            y3 = median5(x31,x32,x33,x34,x35)
            y4 = median5(x41,x42,x43,x44,x45)
            y5 = median5(x51,x52,x53,x54,x55)
            
            z = median5(y1,y2,y3,y4,y5)
            out[i,j] = z
        end
    end
    out
end

medmedpool55! (generic function with 1 method)

In [5]:
using BenchmarkTools
imgs = randn(Float32, 1024,1024, 10)
img = view(imgs, :,:,1)
out = similar(img, size(img) .÷ 5)
@benchmark medmedpool55!(out, img)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.746 ms (0.00% GC)
  median time:      10.786 ms (0.00% GC)
  mean time:        11.430 ms (0.00% GC)
  maximum time:     47.553 ms (0.00% GC)
  --------------
  samples:          437
  evals/sample:     1

In [13]:
size(imgs),size([rand(T,N) for _ in 1:6])

((1024, 1024, 10), (6,))

In [ ]:
Base.@pure simdwidth(::Type{T}) where {T} = Int(256/8/sizeof(T))

@inline function median3(a,b,c)
    max(min(a,b), min(c,max(a,b)))
end

@inline function median5(a,b,c,d,e)
    # https://stackoverflow.com/questions/480960/code-to-calculate-median-of-five-in-c-sharp
    f=max(min(a,b),min(c,d))
    g=min(max(a,b),max(c,d))
    median3(e,f,g)
end

@noinline function median5_vectors!(out, a,b,c,d,e)
    K = simdwidth(eltype(out))
    N = length(out)
    T = eltype(out)
    V = Vec{K,T}
    @assert mod(N,K) == 0

    @inbounds for i in 1:K:N
        va = vload(V,a, i)
        vb = vload(V,b, i)
        vc = vload(V,c, i)
        vd = vload(V,d, i)
        ve = vload(V,e, i)
        vo = median5(va,vb,vc,vd,ve)
        vstore(vo,out, i)
    end
    out
end

using BenchmarkTools
T = UInt8
T = Float32
N = 10^6
N = N ÷ simdwidth(T) * simdwidth(T)
out, a,b,c,d,e = [rand(T,N) for _ in 1:6]
@benchmark median5_vectors!(out, a,b,c,d,e)


In [47]:
simdwidth(Float16)

16

In [127]:
simdwidth(::Type{T}) where {T} = Int(256/8/sizeof(T))



simdwidth (generic function with 1 method)

In [138]:

simdwidth(Float16)

16

In [5]:
struct VecElement{T}
    value::T
end


In [6]:
const m128 = NTuple{4,VecElement{Float32}}

function add(a::m128, b::m128)
    (VecElement(a[1].value+b[1].value),
     VecElement(a[2].value+b[2].value),
     VecElement(a[3].value+b[3].value),
     VecElement(a[4].value+b[4].value))
end

triple(c::m128) = add(add(c,c),c)

code_native(triple,(m128,))


	.section	__TEXT,__text,regular,pure_instructions
Filename: In[6]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 10
	movss	(%rsi), %xmm0           ## xmm0 = mem[0],zero,zero,zero
	movss	4(%rsi), %xmm1          ## xmm1 = mem[0],zero,zero,zero
	movaps	%xmm0, %xmm2
	addss	%xmm2, %xmm2
	movaps	%xmm1, %xmm3
	addss	%xmm3, %xmm3
	movss	8(%rsi), %xmm4          ## xmm4 = mem[0],zero,zero,zero
	movaps	%xmm4, %xmm5
	addss	%xmm5, %xmm5
	movss	12(%rsi), %xmm6         ## xmm6 = mem[0],zero,zero,zero
	movaps	%xmm6, %xmm7
	addss	%xmm7, %xmm7
	addss	%xmm0, %xmm2
	addss	%xmm1, %xmm3
	addss	%xmm4, %xmm5
	addss	%xmm6, %xmm7
	movss	%xmm2, (%rdi)
	movss	%xmm3, 4(%rdi)
	movss	%xmm5, 8(%rdi)
	movss	%xmm7, 12(%rdi)
	movq	%rdi, %rax
	popq	%rbp
	retq
	nopl	(%rax,%rax)
